# Please Note: This is not the final code utilized in the paper!
This is "legacy code" that I am showing as it is part of my innitial steps for this project

In [ ]:
#Initial steps used alot of built in keras functionality as a "sanity check"
import numpy as np
import keras as keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense , Dropout , Activation
from keras.layers import LSTM , Embedding
from keras.layers import MaxPooling1D , GlobalMaxPooling1D,Conv1D , Flatten
from keras.datasets import imdb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
#Take any datasets with a frequency atmost reaching the "top_words" limit
top_words = 8000
(X_train , y_train), (X_test , y_test) = imdb.load_data(num_words = top_words)

#Debug code that is used to convert self created string into arrays useable for testing
print(X_test[0])
imdbDict = keras.datasets.imdb.get_word_index()

inputString = "I think the movie was really good the shots were stunning and the actors did a good job portraying the characters would recommend this is definitely one of the directors better works i am stunned at the amazement"
inputArray = inputString.split()
convertedArray = []
for x in inputArray:
  if (x in imdbDict):
    convertedArray.append(imdbDict[x])
  else:
    print("<",x, "> was not found in the imdb dictionary")

print(convertedArray)



In [ ]:
#The actual model, first pad the sequence, then perform an architecture similar
#to that of an AlexNet except only using 1D convolution and pooling layers

max_words = 450
print(X_train.shape)
x_train = sequence.pad_sequences(X_train , maxlen = max_words)
x_test = sequence.pad_sequences(X_test , maxlen = max_words)
print(x_train.shape)
model = Sequential()
model.add(Embedding(top_words , 32 , input_length = max_words))

model.add(Conv1D(128 , 5 , padding = 'same',activation = 'relu'))
model.add(MaxPooling1D(5))

model.add(Conv1D(128 , 5 , padding = 'same',activation = 'relu'))
model.add(MaxPooling1D(5))

model.add(Conv1D(128 , 5 , padding = 'same',activation = 'relu'))
model.add(MaxPooling1D(5))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
model.summary()

In [ ]:
#Fitting and evaluating

model.fit(x_train , y_train , validation_data= (x_test , y_test) ,batch_size = 128, epochs = 10 , verbose = 2)
scores = model.evaluate(x_test, y_test, verbose=0)


#Additional Evaluation Metrics
y_pred = (y_pred_prob > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

#TEsting of self created sentences
compiledScore = model.predict(sequence.pad_sequences([convertedArray] , maxlen = max_words))[0][0]

print(compiledScore)
if compiledScore < 0.5:
  compiledText = "Negative"
else:
  compiledText = "Positive"

print("Review made:", inputString, ", Compiled Score:", compiledText)